# CNN para Pronóstico de Series Temporales

Este notebook implementa una Red Neuronal Convolucional (CNN) aplicada al pronóstico de series temporales. Se toma como referencia el artículo:

**Deng, Q., et al. "Short-Term Load Forecasting Based on Deep Convolutional Neural Networks and Temporal Attention Mechanism." IEEE Transactions on Smart Grid, 2020.**

El artículo presenta un enfoque en el que se utilizan CNN para capturar patrones locales en los datos temporales, mejorando la precisión en la predicción a corto plazo comparado con métodos tradicionales. A continuación se implementa un ejemplo práctico con datos sintéticos.

## Discusión del Artículo Científico

El artículo de Deng et al. (2020) destaca los siguientes puntos clave:

- **Captura de Patrones Locales:** Las CNN detectan características locales en las series temporales, lo que es crucial para modelar la dinámica de datos.
- **Mejora en la Precisión:** La extracción de características relevantes y la reducción de dimensionalidad permiten obtener un modelo con menor error en la predicción.
- **Interpretabilidad:** La incorporación de mecanismos de atención temporal ayuda a identificar las partes más influyentes de la serie para la predicción, ofreciendo mayor interpretabilidad del modelo.

Esta discusión respalda la implementación siguiente, donde se utiliza un conjunto de datos sintético para demostrar la aplicación de estas técnicas.

## Objetivos del Notebook

- Demostrar la aplicación de CNN en el pronóstico de series temporales.
- Implementar y evaluar un modelo CNN sobre datos sintéticos.
- Comparar los resultados obtenidos con la discusión presentada en el artículo científico.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Generación de Datos

Se genera una serie temporal sintética a partir de una función seno con la adición de ruido aleatorio para simular variaciones en los datos.

In [ ]:
np.random.seed(0)
t = np.arange(0, 100, 0.1)
sinusoid = np.sin(0.1 * t)
noise = np.random.normal(0, 0.1, len(t))
time_series = sinusoid + noise

plt.figure(figsize=(12, 6))
plt.plot(t, time_series)
plt.title('Serie Temporal Sintética: Onda Senoidal con Ruido')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.grid(True)
plt.show()

## Preparación de Datos

Se crean secuencias de 10 pasos consecutivos que servirán como entrada para predecir el siguiente valor en la serie utilizando el enfoque de ventana deslizante.

In [ ]:
n_steps = 10
X, y = [], []
for i in range(len(time_series) - n_steps):
    X.append(time_series[i:i+n_steps])
    y.append(time_series[i+n_steps])
X = np.array(X).reshape(-1, n_steps, 1)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Forma del conjunto de entrenamiento: X_train={X_train.shape}, y_train={y_train.shape}')
print(f'Forma del conjunto de prueba: X_test={X_test.shape}, y_test={y_test.shape}')

## Construcción del Modelo CNN

Se define un modelo CNN que consta de una capa convolucional, seguida de una capa de max pooling, una capa de aplanamiento y dos capas densas. Esta arquitectura permite capturar los patrones locales en la serie temporal y realizar la regresión del valor siguiente.

In [ ]:
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

## Entrenamiento del Modelo

El modelo se entrena durante 50 épocas con un tamaño de batch de 16, utilizando el conjunto de validación para monitorear el desempeño durante el entrenamiento.

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_test, y_test),
    verbose=1
)

plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Pérdida de Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de Validación')
plt.title('Historial de Entrenamiento del Modelo')
plt.xlabel('Época')
plt.ylabel('Pérdida (MSE)')
plt.legend()
plt.grid(True)
plt.show()

## Evaluación del Modelo

Se evalúa el desempeño del modelo en el conjunto de prueba comparando las predicciones con los datos reales y calculando el RMSE (raíz del error cuadrático medio).

In [ ]:
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE en el conjunto de prueba: {rmse}')

plt.figure(figsize=(12, 6))
plt.plot(y_test, label='Datos reales', marker='o')
plt.plot(y_pred, label='Pronóstico', marker='o')
plt.xlabel('Índice de muestra')
plt.ylabel('Valor')
plt.title('Comparación: Datos Reales vs Predicción')
plt.legend()
plt.grid(True)
plt.show()

## Pronósticos Futuros

Utilizando el modelo entrenado, se realizan predicciones para 50 pasos futuros a partir de la última secuencia del conjunto de prueba.

In [ ]:
future_steps = 50
future_predictions = []
last_sequence = X_test[-1].reshape(1, n_steps, 1)

for _ in range(future_steps):
    next_pred = model.predict(last_sequence)[0][0]
    future_predictions.append(next_pred)
    last_sequence = np.roll(last_sequence, -1, axis=1)
    last_sequence[0, -1, 0] = next_pred

plt.figure(figsize=(12, 6))
plt.plot(time_series, label='Serie Temporal Original')
plt.plot(range(len(time_series), len(time_series) + future_steps), future_predictions, 'r--', label='Predicciones Futuras')
plt.title('Serie Temporal con Predicciones Futuras')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.legend()
plt.grid(True)
plt.show()

## Conclusiones

- Las CNN son efectivas para capturar patrones locales en series temporales, lo que se refleja en un mejor desempeño en la predicción.
- El modelo entrenado sobre datos sintéticos mostró un desempeño adecuado, evidenciado por el bajo RMSE en el conjunto de prueba.
- La metodología descrita en el artículo respalda la aplicación de CNN en problemas de pronóstico de series temporales, permitiendo identificar características relevantes y mejorar la interpretabilidad mediante mecanismos de atención.

Este notebook sienta las bases para aplicar técnicas similares en conjuntos de datos reales y comparar los resultados con los obtenidos en la literatura científica.